In [1]:
import pandas as pd

In [2]:
!pip install lightgbm

In [3]:
import lightgbm

In [4]:
data=pd.read_csv("insurance_pre.csv")

In [5]:
data=pd.get_dummies(data,drop_first=True)

In [6]:
data

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [7]:
independent=data[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [8]:
dependent=data[['charges']]

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

grid_param={'learning_rate': [0.1, 0.01], 'n_estimators': [50, 100, 150],
            'max_depth': [4]}
grid=GridSearchCV(LGBMRegressor(),grid_param, refit=True, verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.094701 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01], 'max_depth': [4],
                         'n_estimators': [50, 100, 150]},
             verbose=3)

In [12]:
results=grid.cv_results_
results

{'mean_fit_time': array([26.85470343, 61.15775476, 86.47102432, 39.93273058, 74.82093897,
        99.7301137 ]),
 'std_fit_time': array([0.96187909, 3.97668941, 3.39399818, 0.26024262, 8.02206734,
        7.71186654]),
 'mean_score_time': array([0.03372664, 0.04389415, 0.0301218 , 0.04999676, 0.02499857,
        0.03692403]),
 'std_score_time': array([0.01193409, 0.0181215 , 0.01726296, 0.0318669 , 0.00765368,
        0.01505032]),
 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1, 0.01, 0.01, 0.01],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[4, 4, 4, 4, 4, 4],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[50, 100, 150, 50, 100, 150],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{

In [16]:
ypred=grid.predict(x_test)

In [17]:
from sklearn.metrics import r2_score
rscore=r2_score(y_test,ypred)

In [18]:
print("The Rscore value with best Tuning Parameter is {} :".format(grid.best_params_),rscore)

The Rscore value with best Tuning Parameter is {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50} : 0.8913677409517082


In [19]:
results=grid.cv_results_

In [20]:
table=pd.DataFrame.from_dict(results)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,26.854703,0.961879,0.033727,0.011934,0.1,4,50,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.877935,0.819955,0.839269,0.849329,0.804155,0.838128,0.025282,1
1,61.157755,3.976689,0.043894,0.018122,0.1,4,100,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.873554,0.813261,0.836734,0.843192,0.799561,0.833260,0.025573,2
2,86.471024,3.393998,0.030122,0.017263,0.1,4,150,"{'learning_rate': 0.1, 'max_depth': 4, 'n_esti...",0.869466,0.805203,0.836003,0.836827,0.795245,0.828549,0.026277,3
3,39.932731,0.260243,0.049997,0.031867,0.01,4,50,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",0.556077,0.491364,0.536102,0.543854,0.493347,0.524149,0.026736,6
4,74.820939,8.022067,0.024999,0.007654,0.01,4,100,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",0.765793,0.690124,0.730766,0.745654,0.679928,0.722453,0.032678,5
5,99.730114,7.711867,0.036924,0.015050,0.01,4,150,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",0.840476,0.767490,0.801957,0.816223,0.753411,0.795912,0.031778,4


In [21]:
age_input=float(input("Age : "))
bmi_input=float(input("BMI : "))
child_input=int(input("Children: "))
sex_input=int(input("Sex Male 0 or 1 : "))
smoker_s_input=int(input("Smoker yes 0 or 1 : "))

Age : 33
BMI : 30
Children: 2
Sex Male 0 or 1 : 1
Smoker yes 0 or 1 : 1


In [25]:
future_predict=grid.predict([[age_input,bmi_input,child_input,sex_input,smoker_s_input]])

In [26]:
print("The Future Prediction is{}: ".format(future_predict))

The Future Prediction is[35827.83301431]: 
